# Case Study  - Movies

# Identification Information

# Setup

In [1]:
install.packages('recommenderlab')

Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)
also installing the dependency ‘irlba’



# Import

In [3]:
library('recommenderlab')

In [ ]:
# Data

In [4]:
data = read.table('u.data.txt', col.names = c("user_id", "item_id", "rating", "timestamp") )
data = data[ , -which(names(data) %in% c("timestamp"))]
str(data)

'data.frame':	100000 obs. of  3 variables:
 $ user_id: int  196 186 22 244 166 298 115 253 305 6 ...
 $ item_id: int  242 302 377 51 346 474 265 465 451 86 ...
 $ rating : int  3 3 1 2 1 4 2 5 3 3 ...


In [7]:
summary(data)

    user_id         item_id           rating    
 Min.   :  1.0   Min.   :   1.0   Min.   :1.00  
 1st Qu.:254.0   1st Qu.: 175.0   1st Qu.:3.00  
 Median :447.0   Median : 322.0   Median :4.00  
 Mean   :462.5   Mean   : 425.5   Mean   :3.53  
 3rd Qu.:682.0   3rd Qu.: 631.0   3rd Qu.:4.00  
 Max.   :943.0   Max.   :1682.0   Max.   :5.00  

In [9]:
table(data$rating)


    1     2     3     4     5 
 6110 11370 27145 34174 21201 

In [10]:
mean(data$rating) 

[1] 3.52986

In [29]:
install.packages("ggplot2")
library(ggplot2)


Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)


In [41]:
ggplot(data=data, aes(data$rating)) + 
  geom_histogram( binwidth = 0.5, 
                 fill=I("blue"), 
                 col=I("red"), 
                 alpha=I(.2)) +
labs(title="Histogram of Ratings", x="Rating", y="Count") 

In [6]:
data_new = as(data, "realRatingMatrix")

data_split = evaluationScheme(data_new, method="split", train=0.7, goodRating=3, given=-1)
train = getData(data_split, "train")
test_X = getData(data_split, "known")
test_Y = getData(data_split, "unknown")


We have 100000 observations of movie ratings. The distribution of ratings is positive with most cases related to 4. As we can see in the figures, people tend to rank mostly with 4 and the average rating of the whole sample is 3.52986 .

# Model 1: Random

In [11]:
model_random = Recommender(train, method="RANDOM")

In [12]:
predicted_random = predict(model_random, test_X, type="ratings")
error_random = calcPredictionAccuracy(predicted_random, test_Y)
error_random


RMSE      MSE      MAE 
1.496790 2.240381 1.133659

# Model 2: User-Based Collaborative Filtering

In [13]:
model_user = Recommender(train, method="UBCF")

In [14]:
predicted_user = predict(model_user, test_X, type="ratings")
error_user = calcPredictionAccuracy(predicted_user, test_Y)
error_user

RMSE       MSE       MAE 
1.0178905 1.0361010 0.8235555

# Model 3: Item-Based Collaborative Filtering

In [17]:
model_item = Recommender(train, method="IBCF")

In [18]:
predicted_item = predict(model_item, test_X, type="ratings")
error_item = calcPredictionAccuracy(predicted_item, test_Y)
error_item

RMSE       MSE       MAE 
1.2060454 1.4545455 0.8636364

We compared each method using a test set of observations. The model that perfomed better was the User-Based Collaborative model. It has the lowest RMSE. We could expect that the user and item based models woudld outperform the random model in advance. The user based model take into account user similar preferences in order to recomend items that were recommened by similar users. 

# Model 4: Matrix Factorization

In [19]:
model_matrix = Recommender(train, method="SVD")

In [20]:
predicted_matrix = predict(model_matrix, test_X, type="ratings")
error_matrix = calcPredictionAccuracy(predicted_matrix, test_Y)
error_matrix

RMSE       MSE       MAE 
0.9893670 0.9788471 0.7967531

In [21]:
results_random = evaluate(data_split, method="RANDOM", type="topNList", n=c(5, 10))
results_random@results

RANDOM run fold/sample [model time/prediction time]
	 1  [0.085sec/2.34sec] 


[[1]]
An object of class "confusionMatrix"
Slot "cm":
            TP       FP        FN       TN    precision      recall         TPR
5  0.003533569 4.996466 0.8621908 1677.138 0.0007067138 0.004081633 0.004081633
10 0.007067138 9.992933 0.8586572 1672.141 0.0007067138 0.008163265 0.008163265
           FPR
5  0.002970314
10 0.005940627

Slot "model":
NULL



In [22]:
results_user = evaluate(data_split, method="UBCF", type="topNList", n=c(5, 10))
results_user@results

UBCF run fold/sample [model time/prediction time]
	 1  [0.155sec/6.678sec] 


[[1]]
An object of class "confusionMatrix"
Slot "cm":
           TP       FP        FN       TN   precision     recall        TPR
5  0.02826855 4.971731 0.8374558 1677.163 0.005653710 0.03265306 0.03265306
10 0.03180212 9.968198 0.8339223 1672.166 0.003180212 0.03673469 0.03673469
           FPR
5  0.002955608
10 0.005925922

Slot "model":
NULL



In [23]:
results_item = evaluate(data_split, method="IBCF", type="topNList", n=c(5, 10))
results_item@results

IBCF run fold/sample [model time/prediction time]
	 1  [59.654sec/0.676sec] 


[[1]]
An object of class "confusionMatrix"
Slot "cm":
            TP       FP        FN       TN    precision      recall         TPR
5  0.003533569 4.996466 0.8621908 1677.138 0.0007067138 0.004081633 0.004081633
10 0.003533569 9.996466 0.8621908 1672.138 0.0003533569 0.004081633 0.004081633
           FPR
5  0.002970314
10 0.005942728

Slot "model":
NULL



In [24]:
results_matrix = evaluate(data_split, method="SVD", type="topNList", n=c(5, 10))
results_matrix@results

SVD run fold/sample [model time/prediction time]
	 1  [2.611sec/1.87sec] 


[[1]]
An object of class "confusionMatrix"
Slot "cm":
           TP       FP        FN       TN   precision     recall        TPR
5  0.03180212 4.968198 0.8339223 1677.166 0.006360424 0.03673469 0.03673469
10 0.05300353 9.946996 0.8127208 1672.187 0.005300353 0.06122449 0.06122449
           FPR
5  0.002953507
10 0.005913317

Slot "model":
NULL



The main difference we can find in the SVD model. It has a higher precision and recall compared to the user and itel collaborative models. It show us that despite the good general prediction among all observations it may not perform so well in some special unbalanced cases.

#  Top-`n` Predictions

In [28]:
user_id = 10
n = 10

In [27]:
for (model in c(model_random, model_user, model_item, model_matrix)) {
    # Get recs for model
    user_recs = predict(model, test_X, n=n, type="ratings")
    
    # Print the top n recs for that user
    print(model)
    print(user_recs@data[user_id,][order(-user_recs@data[user_id,])][c(1:n)])
}

Recommender of type ‘RANDOM’ for ‘realRatingMatrix’ 
learned using 660 users.
 3  6 13 17 19 21 24 29 32 34 
 5  5  5  5  5  5  5  5  5  5 
Recommender of type ‘UBCF’ for ‘realRatingMatrix’ 
learned using 660 users.
     313      286      302      272      269      304      258      316 
4.852032 4.766831 4.649340 4.622363 4.588762 4.587095 4.570055 4.564316 
     877      990 
4.543260 4.539852 
Recommender of type ‘IBCF’ for ‘realRatingMatrix’ 
learned using 660 users.
 37  39  57 160 256 360 456 505 537 569 
  5   5   5   5   5   5   5   5   5   5 
Recommender of type ‘SVD’ for ‘realRatingMatrix’ 
learned using 660 users.
     286       22      739       66      451      393      476      660 
4.538444 4.533143 4.526056 4.512577 4.508604 4.507167 4.496858 4.496222 
     258      125 
4.492963 4.492663 


In some cases makes sense but in some models we can see predictions on the extreme values. We can see the rating values in the second row of each model prediction. In the case of prediction to a certain company we could arrange a recomendation list according to this ordered prediction rating list. 